## Vision Transformer (ViT)

In this assignment we're going to work with Vision Transformer. We will start to build our own vit model and train it on an image classification task.
The purpose of this homework is for you to get familar with ViT and get prepared for the final project.

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# VIT Implementation

The vision transformer can be seperated into three parts, we will implement each part and combine them in the end.

For the implementation, feel free to experiment different kinds of setup, as long as you use attention as the main computation unit and the ViT can be train to perform the image classification task present later.
You can read about the ViT implement from other libary: https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py and https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py

## PatchEmbedding
PatchEmbedding is responsible for dividing the input image into non-overlapping patches and projecting them into a specified embedding dimension. It uses a 2D convolution layer with a kernel size and stride equal to the patch size. The output is a sequence of linear embeddings for each patch.

In [3]:
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
      super().__init__()
      self.image_size = image_size
      self.patch_size = patch_size
      self.in_channels = in_channels
      self.embed_dim = embed_dim
      self.grid_size = image_size // patch_size
      self.num_patches = self.grid_size ** 2
      self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)


    def forward(self, x):
      x = self.proj(x).flatten(2).transpose(1, 2)
      return x

## MultiHeadSelfAttention

This class implements the multi-head self-attention mechanism, which is a key component of the transformer architecture. It consists of multiple attention heads that independently compute scaled dot-product attention on the input embeddings. This allows the model to capture different aspects of the input at different positions. The attention outputs are concatenated and linearly transformed back to the original embedding size.

In [4]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
      super().__init__()
      self.embed_dim = embed_dim
      self.num_heads = num_heads
      self.head_dim = embed_dim // num_heads

      self.query = nn.Linear(embed_dim, embed_dim)
      self.key = nn.Linear(embed_dim, embed_dim)
      self.value = nn.Linear(embed_dim, embed_dim)
      self.scale = self.head_dim ** -0.5

    def forward(self, x):
      B, N, embed_dim = x.shape

      queries = self.query(x)
      keys = self.key(x)
      values = self.value(x)

      queries = queries.view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
      keys = keys.view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
      values = values.view(B, N, self.num_heads, self.head_dim).transpose(1, 2)

      attention_scores = torch.matmul(queries, keys.transpose(-1, -2)) * self.scale
      # print(attention_scores.shape)
      attention_weights = F.softmax(attention_scores, dim=-1)
      attention_output = torch.matmul(attention_weights, values)

      attention_output = attention_output.transpose(1, 2).contiguous().view(B, N, self.embed_dim)

      return attention_output

## TransformerBlock
This class represents a single transformer layer. It includes a multi-head self-attention sublayer followed by a position-wise feed-forward network (MLP). Each sublayer is surrounded by residual connections.
You may also want to use layer normalization or other type of normalization.

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
      super().__init__()
      self.norm1 = nn.LayerNorm(embed_dim)
      self.norm2 = nn.LayerNorm(embed_dim)
      self.self_attention = MultiHeadSelfAttention(embed_dim, num_heads)

      self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim),
            nn.Dropout(dropout)
        )

      self.dropout = nn.Dropout(dropout)

    def forward(self, x):
      x = x + self.dropout(self.self_attention(x))
      x = self.norm1(x)
      x = x + self.dropout(self.mlp(x))
      x = self.norm2(x)
      return x

## VisionTransformer:
This is the main class that assembles the entire Vision Transformer architecture. It starts with the PatchEmbedding layer to create patch embeddings from the input image. A special class token is added to the sequence, and positional embeddings are added to both the patch and class tokens. The sequence of patch embeddings is then passed through multiple TransformerBlock layers. The final output is the logits for all classes

In [6]:
class VisionTransformer(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout=0.1):
      super().__init__()
      self.patch_embed = PatchEmbedding(image_size, patch_size, in_channels, embed_dim)
      self.num_patches = (image_size // patch_size) ** 2
      self.class_token = nn.Parameter(torch.randn(1, 1, embed_dim))
      self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, embed_dim))
      self.transformer_blocks = nn.ModuleList([
          TransformerBlock(embed_dim, num_heads, mlp_dim, dropout)
          for _ in range(num_layers)
      ])
      self.mlp_head = nn.Sequential(
          nn.LayerNorm(embed_dim),
          nn.Linear(embed_dim, num_classes)
      )
      self.dropout = nn.Dropout(dropout)


    def forward(self, x):
      B = x.shape[0]
      x = self.patch_embed(x)
      class_token = self.class_token.expand(B, -1, -1)
      x = torch.cat((class_token, x), dim=1)
      x = x + self.pos_embed
      x = self.dropout(x)
      for transformer_block in self.transformer_blocks:
          x = transformer_block(x)
      class_output = x[:, 0]
      logits = self.mlp_head(class_output)
      return logits


## Let's train the ViT!

We will train the vit to do the image classification with cifar100. Free free to change the optimizer and or add other tricks to improve the training

In [7]:
# Example usage:
image_size = 32
patch_size = 4
in_channels = 3
embed_dim = 256
num_heads = 8
mlp_dim = 512
num_layers = 6
num_classes = 100
dropout = 0.1

batch_size = 1024

In [8]:
model = VisionTransformer(image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout).to(device)
input_tensor = torch.randn(1, in_channels, image_size, image_size).to(device)
print(f"input: {input_tensor.shape}")
output = model(input_tensor)
print(output.shape)

input: torch.Size([1, 3, 32, 32])
torch.Size([1, 100])


In [9]:
# Load the CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 169001437/169001437 [00:03<00:00, 42893967.57it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [10]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [11]:
# Train the model
num_epochs = 100
best_val_acc = 0
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # TODO Feel free to modify the training loop youself.

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Epoch: {epoch + 1}, Validation Accuracy: {val_acc:.2f}%")

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

Epoch: 1, Validation Accuracy: 8.43%
Epoch: 2, Validation Accuracy: 15.31%
Epoch: 3, Validation Accuracy: 20.93%
Epoch: 4, Validation Accuracy: 25.20%
Epoch: 5, Validation Accuracy: 27.24%
Epoch: 6, Validation Accuracy: 29.74%
Epoch: 7, Validation Accuracy: 32.30%
Epoch: 8, Validation Accuracy: 34.03%
Epoch: 9, Validation Accuracy: 37.51%
Epoch: 10, Validation Accuracy: 39.05%
Epoch: 11, Validation Accuracy: 38.95%
Epoch: 12, Validation Accuracy: 41.14%
Epoch: 13, Validation Accuracy: 42.77%
Epoch: 14, Validation Accuracy: 44.26%
Epoch: 15, Validation Accuracy: 44.37%
Epoch: 16, Validation Accuracy: 46.10%
Epoch: 17, Validation Accuracy: 47.02%
Epoch: 18, Validation Accuracy: 47.88%
Epoch: 19, Validation Accuracy: 48.38%
Epoch: 20, Validation Accuracy: 48.36%
Epoch: 21, Validation Accuracy: 49.86%
Epoch: 22, Validation Accuracy: 49.68%
Epoch: 23, Validation Accuracy: 50.29%
Epoch: 24, Validation Accuracy: 50.19%
Epoch: 25, Validation Accuracy: 51.68%
Epoch: 26, Validation Accuracy: 51.

Please submit your best_model.pth with this notebook. And report the best test results you get.

The best result I got is an accuracy of 54.20% on the validation set at epoch 85.

In [14]:
torch.save(model, "/dl/assignment3/best_model.pth")